In [1]:
from __future__ import print_function, division
from gurobipy import Model, GRB, quicksum, LinExpr
import networkx as nx
import pickle
import numpy, random

In [2]:
def MIP_IM():
    attributes = ['gender']#, 'gender', 'age', 'region', 'ethnicity']
  
    budget = 25
    m = 100
    for attribute in attributes:
        for index in range(20):
            with open('../MIP/data/networks_prob/graph_spa_500_'+str(index)+'.pickle', "rb") as f:
                main_graph = pickle.load(f)

                samples = []
                for j in range(m):
                    G = nx.DiGraph()
                    for u in main_graph.nodes():
                        G.add_node(u)
                    for u,v in main_graph.edges():
                        if main_graph[u][v]['p']> random.random():
                            G.add_edge(u, v)
                    samples.append(G)

                labels = nx.get_node_attributes(main_graph, attribute)
                label_dict = {}
                for i in range(len(main_graph.nodes())):
                    label = labels[i].encode('utf-8')
                    if label in label_dict.keys():
                        label_dict[label].append(i)
                    else:
                        label_dict[label] = [i] 
                        
                model = Model('mean_'+str(attribute)+'_'+str(index))
                min_value = model.addVar(lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS)


                mvars = []
                #active nodes
                avars = []
                #seed nodes
                svars = []
                var_seed_dict = {}
                var_active_dict = {}
                var_mean_dict = {}

                for j in range(len(main_graph.nodes())):
                    s = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                    svars.append(s)
                    var_seed_dict[j] = s

                for sample_index, sample in enumerate(samples):
                    for j in range(len(main_graph.nodes())):
                        a = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                        avars.append(a)
                        var_active_dict[(sample_index,j)] = a    

                mvars.append(avars)
                mvars.append(svars)

                obj_expr = quicksum(avars)
                model.setObjective(min_value, GRB.MAXIMIZE)

                model.addConstr(quicksum(svars), GRB.LESS_EQUAL, budget)

                for sample_index, sample in enumerate(samples):
                    for i in range(len(main_graph.nodes())):
                        neighbors = sample.in_edges(i) 
                        e = len(neighbors)
                        ai = var_active_dict[(sample_index,i)]
                        si = var_seed_dict[i]
                        if i in  var_mean_dict.keys():
                            var_mean_dict[i]+= ai
                        else:
                            var_mean_dict[i]= ai
                        neighbors_vars = []
                        for neighbor in neighbors:
                            neighbors_vars.append(var_active_dict[(sample_index,neighbor[0])])
                        expr = (e+1)*ai-si-quicksum(neighbors_vars)
                        model.addConstr(expr, GRB.GREATER_EQUAL, 0)
                        model.addConstr(expr, GRB.LESS_EQUAL, e)

                mean_label_dict = {}
                for label in label_dict.keys():
                    for node in label_dict[label]:
                        if label in mean_label_dict.keys():
                            mean_label_dict[label]+=var_mean_dict[node]
                        else:
                            mean_label_dict[label]=var_mean_dict[node]
        
        
                for label in label_dict.keys():
                    label_size = len(label_dict[label])
                    model.addConstr(mean_label_dict[label], GRB.GREATER_EQUAL, label_size *m* min_value)

                try:
                    model.optimize()
                except e:
                    print(e)

                with open('../Git/influence_maximization/experiments/im500/results/fairmip/mean/base100/'+attribute+'/output_'+str(index)+'.txt', "w") as of:    
                    for key,value in var_seed_dict.items():
                        if(value.x > 0):
                            print(key, file=of)

In [3]:
MIP_IM()

Optimize a model with 100003 rows, 50501 columns and 299010 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 64172 rows and 32166 columns
Presolve time: 2.04s
Presolved: 35831 rows, 18335 columns, 130122 nonzeros
Found heuristic solution: objective 0.0526275
Variable types: 1 continuous, 18334 integer (18334 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolved: 35831 rows, 18335 columns, 130122 nonzeros


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   34648    2.1188438e-01   0.000000e+00   1.937130e-01      5s
   36250    2.1946347e-01   0.000000e+00   0.000000e+00      5s
Concurrent spin time: 0.08s

Solved with primal simplex

Root relaxation: objec

     0     0    0.21389    0  292    0.21371    0.21389  0.08%     -    7s
     0     0    0.21389    0    3    0.21371    0.21389  0.08%     -    8s
     0     0    0.21382    0    4    0.21371    0.21382  0.05%     -    8s
     0     0    0.21382    0    2    0.21371    0.21382  0.05%     -    8s
H    0     0                       0.2137959    0.21382  0.01%     -    8s
     0     0    0.21382    0    3    0.21380    0.21382  0.01%     -    8s

Cutting planes:
  Gomory: 1
  MIR: 1
  GUB cover: 1

Explored 1 nodes (44285 simplex iterations) in 8.46 seconds
Thread count was 8 (of 8 available processors)

Solution count 4: 0.213796 0.213714 0.213137 0.0583265 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.137959183673e-01, best bound 2.137959183673e-01, gap 0.0000%
Optimize a model with 100003 rows, 50501 columns and 298522 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+

  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 63717 rows and 31934 columns
Presolve time: 1.98s
Presolved: 36286 rows, 18567 columns, 131937 nonzeros
Found heuristic solution: objective 0.0548800
Variable types: 1 continuous, 18566 integer (18566 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolved: 36286 rows, 18567 columns, 131937 nonzeros


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   29113    1.8958030e-01   0.000000e+00   1.915131e-01      5s
Concurrent spin time: 1.46s

Solved with dual simplex

Root relaxation: objective 2.186566e-01, 22609 iterations, 4.52 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.21866    0  293    0.05488    0.21866   298%     -    6s
H    0   

     0     0    0.20922    0   35    0.20899    0.20922  0.11%     -    8s
     0     2    0.20922    0   35    0.20899    0.20922  0.11%     -    8s
*  108    44              11       0.2090123    0.20919  0.08%  11.6    8s
H  432     2                       0.2090535    0.20908  0.01%   9.0    8s

Cutting planes:
  Gomory: 1
  Cover: 4
  MIR: 3
  Inf proof: 1

Explored 448 nodes (44327 simplex iterations) in 8.61 seconds
Thread count was 8 (of 8 available processors)

Solution count 7: 0.209053 0.209012 0.208988 ... 0.0524903

Optimal solution found (tolerance 1.00e-04)
Best objective 2.090534979424e-01, best bound 2.090557412679e-01, gap 0.0011%
Optimize a model with 100003 rows, 50501 columns and 299300 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve

     0     0    0.21256    0   15    0.21253    0.21256  0.02%     -   11s
     0     0    0.21256    0   16    0.21253    0.21256  0.01%     -   11s
     0     0    0.21255    0   14    0.21253    0.21255  0.01%     -   11s
     0     0    0.21255    0   14    0.21253    0.21255  0.01%     -   11s
     0     2    0.21255    0   14    0.21253    0.21255  0.01%     -   11s

Cutting planes:
  Gomory: 1
  MIR: 1
  StrongCG: 1
  Zero half: 2

Explored 10 nodes (42491 simplex iterations) in 11.55 seconds
Thread count was 8 (of 8 available processors)

Solution count 10: 0.212528 0.212511 0.212415 ... 0.157434

Optimal solution found (tolerance 1.00e-04)
Best objective 2.125283018868e-01, best bound 2.125471198943e-01, gap 0.0089%
Optimize a model with 100003 rows, 50501 columns and 299040 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range

     0     0    0.21901    0  123    0.21872    0.21901  0.14%     -   12s
     0     0    0.21900    0  123    0.21872    0.21900  0.13%     -   12s
     0     0    0.21897    0  159    0.21872    0.21897  0.12%     -   12s
     0     2    0.21897    0  159    0.21872    0.21897  0.12%     -   12s

Cutting planes:
  Cover: 1
  MIR: 2
  Inf proof: 4

Explored 277 nodes (55756 simplex iterations) in 13.09 seconds
Thread count was 8 (of 8 available processors)

Solution count 7: 0.218717 0.218426 0.21834 ... 0.0597872

Optimal solution found (tolerance 1.00e-04)
Best objective 2.187169811321e-01, best bound 2.187169811321e-01, gap 0.0000%
Optimize a model with 100003 rows, 50501 columns and 299176 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 640

     0     0    0.21732    0  115    0.21710    0.21732  0.10%     -   10s
     0     2    0.21731    0  115    0.21710    0.21731  0.10%     -   10s
H   31    15                       0.2172201    0.21731  0.04%  11.0   10s

Cutting planes:
  Cover: 1
  MIR: 5
  StrongCG: 1

Explored 68 nodes (49319 simplex iterations) in 11.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 6: 0.21722 0.217095 0.21695 ... 0.0615444

Optimal solution found (tolerance 1.00e-04)
Best objective 2.172200772201e-01, best bound 2.172200772201e-01, gap 0.0000%
Optimize a model with 100003 rows, 50501 columns and 299232 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 64474 rows and 32308 columns
Presolve time: 2.14s
Presolved: 35529 rows, 18193 co